In [1]:
import scanpy as scp
import scipy.sparse as sprs
import pandas as pd
import warnings
import os
import numpy as np

warnings.filterwarnings("ignore")

from parameters import *

In [2]:
adataRaw = scp.read("Data/Raw.h5ad")

# Counts

In [3]:
d = pd.DataFrame(np.array(adataRaw.X.todense()))
d.to_csv(DIRECTORY+"Results/Counts_2021.csv")

# Metadata

In [4]:
metadataRaw = adataRaw.obs

In [5]:
metadata = scp.read("Analysis_Main/Results/QC.h5ad").obs
metadataSorted = scp.read("Analysis_Main/Results/QC_sorted.h5ad").obs
metadaTime = []
for condition in metadata["Time"].unique():
    metadaTime.append(scp.read("Analysis_Main/Results/QC_"+condition+".h5ad").obs)

In [6]:
X_umap = scp.read("Analysis_Main/Results/QC.h5ad").obsm["X_umap"]
X_umap_sorted = scp.read("Analysis_Main/Results/QC_sorted.h5ad").obsm["X_umap"]
X_umap_time = []
for condition in metadata["Time"].unique():
    X_umap_time.append(scp.read("Analysis_Main/Results/QC_"+condition+".h5ad").obsm["X_pca"])

In [7]:
X_pca = scp.read("Analysis_Main/Results/QC.h5ad").obsm["X_pca"]
X_pca_sorted = scp.read("Analysis_Main/Results/QC_sorted.h5ad").obsm["X_pca"]
X_pca_time = []
for condition in metadata["Time"].unique():
    X_pca_time.append(scp.read("Analysis_Main/Results/QC_"+condition+".h5ad").obsm["X_pca"])

In [12]:
metadataRaw["QC_passed"] = False
metadataRaw.loc[metadata.index,"QC_passed"] = True
metadataRaw.loc[metadata.index,"leiden"] = metadata.loc[:,"leiden"]
metadataRaw.loc[metadataSorted.index,"leiden"] = metadataSorted.loc[:,"leiden"]
for condition in metadaTime:
    metadataRaw.loc[condition.index,"leiden_by_condition"] = condition.loc[:,"leiden"]
    
for i in range(2):
    metadataRaw.loc[metadata.index,"X_umap_"+str(i)] = X_umap[:,i]
for i in range(2):
    metadataRaw.loc[metadataSorted.index,"X_umap_"+str(i)] = X_umap_sorted[:,i]
for condition,umap in zip(metadaTime,X_umap_time):
    for i in range(2):
        metadataRaw.loc[condition.index,"X_umap_by_condition_"+str(i)] = umap[:,i]
        
for i in range(PCA):
    metadataRaw.loc[metadata.index,"X_pca_"+str(i)] = X_pca[:,i]
for i in range(PCA_SORTED):
    metadataRaw.loc[metadataSorted.index,"X_pca_"+str(i)] = X_pca_sorted[:,i]
for condition,umap in zip(metadaTime,X_pca_time):
    for i in range(PCA_TIMES):
        metadataRaw.loc[condition.index,"X_pca_by_condition_"+str(i)] = umap[:,i]

In [13]:
metadataRaw.to_csv(DIRECTORY+"Results/Cells_2021.csv")

# Metadata genes

In [14]:
genes = scp.read("Data/Raw.h5ad").var.drop("Unnamed: 0",axis=1).reset_index()

In [15]:
genes.to_csv(DIRECTORY+"Results/Genes_2021.csv")

In [ ]:
metadataRaw

,Cell,Plate,Time,Condition,BiologicalSample,Flow Cytommetry File,FSC-A,FSC-H,FSC-W,SSC-A,...,X_pca_by_condition_0,X_pca_by_condition_1,X_pca_by_condition_2,X_pca_by_condition_3,X_pca_by_condition_4,X_pca_by_condition_5,X_pca_by_condition_6,X_pca_by_condition_7,X_pca_by_condition_8,X_pca_by_condition_9
0,HG3JFDRXY_IDT-DUI-NXT-193,P1,120hr,whole,P1,P1_120hr_whole_1,78.406,47.042,109.231,41.699,...,19.856091,-1.571342,-12.674532,9.325572,-11.979005,11.533540,-5.601441,-11.412673,1.902156,7.374291
1,HG3JFDRXY_IDT-DUI-NXT-194,P1,120hr,whole,P1,P1_120hr_whole_1,56.736,40.459,91.902,21.335,...,5.040658,-3.539681,-7.943509,2.211347,-0.351983,-7.075374,4.116924,-1.812166,-4.102750,1.349560
2,HG3JFDRXY_IDT-DUI-NXT-195,P1,120hr,whole,P1,P1_120hr_whole_1,57.739,41.270,91.688,23.239,...,-6.868503,-0.066493,2.010788,-4.160384,-2.290377,-3.254133,1.364146,-3.098816,-1.404601,5.102631
3,HG3JFDRXY_IDT-DUI-NXT-196,P1,120hr,whole,P1,P1_120hr_whole_1,74.584,47.043,103.903,31.026,...,-0.667546,-1.552754,-1.171297,0.227236,2.454896,-0.218024,4.392505,0.147907,3.262300,6.936546
4,HG3JFDRXY_IDT-DUI-NXT-197,P1,120hr,whole,P1,P1_120hr_whole_1,80.637,47.131,112.127,38.043,...,2.051070,21.488993,3.781565,-1.690832,-6.246963,6.097060,-11.211215,21.342340,5.297934,-2.233744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,HG72GDRXY_IDT-DUI-NXT-95,P2,144hr,whole,P2,P2_144hr_whole_0,71.227,46.308,100.801,37.759,...,-0.288665,-0.750054,-4.000737,0.801719,-2.558784,0.624562,0.553386,2.042521,1.491300,0.528650
854,HG72GDRXY_IDT-DUI-NXT-96,P2,144hr,whole,empty,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,HG72GDRXY_IDT-DUI-NXT-97,P4,144hr,kitScal,P1,P4_144hr_kitSca1_0,60.811,41.873,95.176,31.348,...,-3.166752,0.287513,6.225529,0.408786,2.695564,-0.496297,-3.245149,2.393794,2.740761,2.500268
856,HG72GDRXY_IDT-DUI-NXT-98,P4,144hr,kitScal,P1,P4_144hr_kitSca1_0,59.391,41.169,94.543,40.658,...,-2.629843,-0.077458,3.361966,0.737768,-0.026165,3.022533,-1.559386,3.418287,0.880544,-0.554813


In [22]:
9*96

864